---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    
    import re
    
    listUtowns = []
    current_state = ''
    town = ''
    with open('university_towns.txt') as fUtowns:
        for line in fUtowns:
            line = line.strip()
            state_line = re.search('(^.*)(\[edit\])$', line)
            if state_line:
                current_state = state_line.group(1)
                #print(current_state)
            else:
                town_line = re.search('(^.*?) \(', line)
                if town_line:
                    town = town_line.group(1)
                else:
                    town = line
                listUtowns.append([current_state, town])
    
    return pd.DataFrame(listUtowns, columns=['State', 'RegionName'])

#get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
def get_recession_stats():
    '''Returns a dict with the starting, ending and bottom quarter
    of the recession as string values in a format such as 2005q3'''
    
    # read the two columns with Quarterly GDP
    GDP = pd.read_excel('gdplev.xls', skiprows=7, parse_cols='E,G', names=['qtr', 'gdp'])
    
    # eliminate rows prior 2000
    GDP = GDP[GDP['qtr'] >= '2000q1'].reset_index(drop=True)

    # create a temporary series with q-to-q delta and merge it in
    delta = GDP['gdp'].diff()
    GDP = (pd.merge(GDP, pd.DataFrame(delta), how='inner', left_index=True, right_index=True)
             .rename(columns={'gdp_x': 'gdp', 'gdp_y': 'p1'})) 

    # remove the first quarter from the temp series, re-index to 0, and merge
    # now we have the delta fro the two prior quarters in each row
    delta = delta.drop(0).reset_index(drop=True)
    GDP = (pd.merge(GDP, pd.DataFrame(delta), how='inner', left_index=True, right_index=True)
             .rename(columns={'gdp_x': 'gdp', 'gdp_y': 'p2'})) 
   
    # find the first row where deltas are both negative
    recession_start = GDP[(GDP['p1'] < 0) & (GDP['p2'] < 0)].iloc[0]
    recession_start = recession_start['qtr']
    
    # find the 2nd quarter followin the start with positive GDP
    recession_end = GDP[(GDP['qtr'] > recession_start) & (GDP['p1'] > 0) & (GDP['p2'] > 0)].iloc[1]
    recession_end = recession_end['qtr']
    
    # isolate the quarters between start and end, and index by quarter
    rdf = GDP[(GDP['qtr'] > recession_start) & (GDP['qtr'] < recession_end)].set_index('qtr')

    # create a dict of the the three stats to return
    stats = {'start': recession_start, 'end': recession_end, 'bottom': rdf['gdp'].idxmin()} 

    return stats


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    stats = get_recession_stats()
    return stats['start']

#get_recession_start()

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    stats = get_recession_stats()
    return stats['end']

#get_recession_end()

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    stats = get_recession_stats()
    return stats['bottom']

#get_recession_bottom()

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    # read the file, drop un-needed cols, fix the state names,
    # and index by state and reqion
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df = (df.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1)
            .replace(to_replace={'State': states})
            .set_index(['State', 'RegionName'])
         )
    
    # transpose, and eliminate rows prior to 2000 Q1
    df = df.transpose()
    df = df[df.index >= '2000-01']

    # convert the index to datetime so we can resample months -> quarters
    df.index = pd.to_datetime(df.index)
    df = df.resample('Q').mean()
    
    # transpost back to orignal (loc X period), and fix the multi-index
    df = (df.transpose()
            .reset_index()
            .sort_values(['State', 'RegionName'])
            .set_index(['State', 'RegionName']))
    
    # turn the columns into quarters
    df.columns = [pd.Period(x, 'Q') for x in df.columns]
        
    return df

#convert_housing_data_to_quarters().shape


In [14]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # call the functions and get the start and end Qs as pd.Periods
    recession_start = pd.Period(pd.to_datetime(get_recession_start()), 'Q') - 1
    recession_bottom = pd.Period(pd.to_datetime(get_recession_bottom()), 'Q')
    
    # call the function to get the housing data and remove empty rows
    housing_data = convert_housing_data_to_quarters().dropna()
    
    # project the columns for the start and end quarter and rename them
    # append a column that shows the change
    # NOTE: the join below won't work if the columns are Periods (so I rename)
    # price_ratio=quarter_before_recession/recession_bottom) 
    housing_data = housing_data[[recession_start, recession_bottom]]
    housing_data.columns = ['SP_START', 'SP_BOTTOM']
    housing_data['PRICE_RATIO'] = housing_data['SP_START'] / housing_data['SP_BOTTOM']
    
    # call the function to get the list of utown
    # set the index to be the same as the housing data
    # add and indicator column and initialize to True
    university_towns = get_list_of_university_towns().set_index(['State', 'RegionName'])
    university_towns['IsUTown'] = True
    
    # do a left merge all of the housing data
    # for the rows that are not U towns set the indicator to False
    df = pd.merge(housing_data, 
                   university_towns, 
                   how='left', 
                   left_index=True, right_index=True).fillna(False)
    
    # split into two dataframes based on the indicator
    UTowns = df[df['IsUTown'] == True]
    OTowns = df[df['IsUTown'] == False]
    better = 'university town'
    if UTowns['PRICE_RATIO'].mean() > OTowns['PRICE_RATIO'].mean():
        better = 'non-university town'
    

    # run the ttest
    ttest = ttest_ind(UTowns['PRICE_RATIO'], OTowns['PRICE_RATIO'])
    
    result = (ttest[1] < 0.01,
              ttest[1],
              better)

    return result #(UTowns['PRICE_RATIO'].mean(), OTowns['PRICE_RATIO'].mean()) 
    
#run_ttest()


(True, 0.0051648663279190233, 'university town')